In [63]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML
from difflib import SequenceMatcher

In [64]:
## get data in file to fulldata[]
fnames = os.listdir("workdata/GOLD_SENT_WITH_POS/")
fnames_ = fnames
fulldata = []
for fname in fnames :
    fr = open("workdata/GOLD_SENT_WITH_POS/" + fname ,encoding = 'utf-8' ,mode ='r')
    lines = fr.readlines()
    fulldata.append(lines)
    fr.close()

In [65]:
os.makedirs('workdata/lines_submit/wrong_lines/sorteddata/', exist_ok=True)
os.makedirs('workdata/lines_submit/correct_lines/sorteddata/', exist_ok=True)
names = ["workdata/lines_submit/wrong_lines/","workdata/lines_submit/correct_lines/"]
for name in names :
#     continue
    fnames = os.listdir(name+"output/")
    for fname in fnames:
        if fname == ".ipynb_checkpoints":
            continue
        path = name + "sorteddata/"+fname[:-3]+"/"
        os.makedirs(path, exist_ok=True)
        fw1 = open(path + "original_line.txt",encoding = 'utf-8' ,mode ='w')
        fw2 = open(path + "input_line.txt",encoding = 'utf-8' ,mode ='w')
        wis = []
        prob = 0
        with open(name+"output/"+fname,encoding = 'utf-8' ,mode ='r') as fr :
            for line in fr.readlines():
                if(line.startswith("filename : ")) :
                    line1 = line
                    continue
                if(line.startswith("lineno : ")) :
                    line2 = line
                    continue
                if(line.startswith("input line : ")) :
                    line3 = line
                    continue
                if(line.startswith("converted line : ")) :
                    line4 = line
                    continue
                if(line.startswith("converted line char :")) :
                    line5 = line
                    continue    
                if(line.startswith("problem_")) :
                    if(line.startswith("problem_yes ,index = ")):
                        line6 = line
                        prob =1

                        line6 = line6[22:-2]
                        for wi in line6.split("-"):
                            wis.append(int(wi))
                        break
                              
        df = pd.read_csv(name+"output/"+fname,encoding = 'utf-8' ,sep = ',',skiprows = 6)
        if(prob==1):
            words = []
            for wi in wis:
                words.append(df.loc[wi,'word'])
            fw3 = open(path + "problem_words.txt",encoding = 'utf-8' ,mode ='w')
            fw3.writelines("undifined_words \n")
            for word in words :
                fw3.writelines(word+"\n")
            fw3.close()
        
        
        fw1.writelines(line1)
        fw1.writelines(line2)
        
        fw2.writelines(line3)
        fw2.writelines(line4)
        fw2.writelines(line5)
        
        df.to_csv(path + "dataframe.txt" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
        
        edf = pd.DataFrame(index=range(0,len(df)))
        for i in edf.index :
            edf[str(i)] = -1
        edf.to_csv(path + "conflicts.csv" ,encoding = 'utf-8' , sep=',',index = False,mode = 'w' )
        
        index = fnames_.index(line1[11:-1])
        lineno = int(line2[9:-1])
        i = 0
        y = 0 
        a = 0
        for lin in fulldata[index]:
            i+= 1
            if(i == lineno ) :
                y=1
                fw1.writelines(lin)
                continue
            if(y==1):
                if(re.match(r'ANALYSIS:',lin)):
                    a=1
                    fw1.writelines(lin)
                    continue
                elif(re.match(r'\d+',lin)) :
                    fw1.writelines(lin)
                    continue
                elif(a==0):
                    y=1
                else :
                    y =0
                    a=0   

        
        fw1.close()
        fw2.close()
        
        
       
        
#         break
#     break

In [66]:
# edf = pd.DataFrame(index=range(0,len(df)))
# for i in edf.index :
#     edf[str(i)] = -1
# for i in df.index
# df

In [67]:
os.makedirs('workdata/lines_submit/wronginput/sorteddata/', exist_ok=True)

with open("workdata/lines_submit/wronginput/output/wronginput.txt",encoding = 'utf-8' ,mode ='r') as fr :
    for line in fr.readlines():
        if(line.startswith("filename : ")) :
            line1 = line
            continue
        if(line.startswith("lineno : ")) :
            line2 = line
            continue
        if(line.startswith("count =")) :
            count = int(line[7:])
            path = "workdata/lines_submit/wronginput/sorteddata/"+str(count)+"/"
            os.makedirs(path, exist_ok=True)
            fw1 = open(path + "original_line.txt",encoding = 'utf-8' ,mode ='w')
            fw2 = open(path + "input_line.txt",encoding = 'utf-8' ,mode ='w')
            fw1.writelines(line1)
            fw1.writelines(line2)
            fw2.writelines("inputline : "+line3 )
            index = fnames_.index(line1[11:-1])
            lineno = int(line2[9:-1])
            i = 0
            y = 0 
            a=0
            for lin in fulldata[index]:
                i+= 1
                if(i == lineno ) :
                    y=1
                    fw1.writelines(lin)
                    continue
                if(y==1):
                    if(re.match(r'ANALYSIS:',lin)):
                        a=1
                        fw1.writelines(lin)
                        continue
                    elif(re.match(r'\d+',lin)) :
                        fw1.writelines(lin)
                        continue
                    elif(a==0):
                        y=1
                    else :
                        y =0
                        a=0
            fw1.close()
            fw2.close()
        else :
            line3 = line

In [43]:
s = pd.read_csv("all_sandhi.txt",encoding = 'utf-8' ,sep = ',')
s.loc[(s['c1']=='A' )& (s['c2']=='A')]
s.loc[s['c1']=='H']

id c1 c2   c3
2557  2557  H  k  ><k
2558  2558  H  k  H k
2559  2559  H  K  ><K
2560  2560  H  K  H K
2561  2561  H  p  ><p
2562  2562  H  p  H p
2563  2563  H  P  ><P
2564  2564  H  P  H P